In [0]:
%run ./EDP_Config

In [0]:
def tblExists(tbl):
    try:
        qry=f"SELECT * FROM {tbl}"
        spark.sql(qry)
    except:
        return False;
    else: 
        return True;


In [0]:
def load_table(df_cfg):
    try:
        tbl_nm = df_cfg['entity_nm']
        tbl_brnz_nm = f'edp_dev.{tbl_nm}_brnz'
        table_slv = f'edp_dev.{tbl_nm}_slv'
        SourceColumnKey = df_cfg['src_clm_key']
        rule_qry = df_cfg['rule_qry']
        upperbound = df_cfg['wtr_mrk_clm_val']
        LdSrtTimenrws = datetime.now().isoformat()

        # Fetching the watermark
        wtrmrkupqry = f"""SELECT wtr_mrk_clm_val FROM edp_dev.incremental_load_cfg WHERE entity_nm='{tbl_nm}'"""
        wtrmrk = spark.sql(wtrmrkupqry).collect()[0][0]

        # Fetching bronze data
        df_brnz = spark.sql(f"""SELECT * FROM {tbl_brnz_nm}""")
        initial_row_count = df_brnz.count()
        if df_brnz.isEmpty():
            MsgDescnorws=f'No rows exists for the table : {table_slv}'
            WorkerCount=worker_count
            LdEndTimenrws=datetime.now().isoformat()
            log_qry_no_rws = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)VALUES (null, '{tbl_nm}', null,null, 0, 'Failure', 0,null,null,null,'{LdSrtTimenrws}','{LdEndTimenrws}',null,'slv','{wtrmrk}', '{LdEndTimenrws}')"""
            spark.sql(log_qry_no_rws)

        if not tblExists(table_slv):
            df_brnz = spark.sql(rule_qry)
            df_brnz.write.format("delta").mode("overwrite").option("overwriteSchema", "True").saveAsTable(table_slv)
            cleaned_row_count =  df_brnz.count()
            ExecutionStatus='Success'
            LdEndTimenrws=datetime.now().isoformat()
            DataRead=df_brnz.count()
            RowsCopied=DataRead
                ##Log message in the log table
            log_qry_no_rws = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)VALUES (null, '{tbl_nm}', null,null, {DataRead},'{ExecutionStatus}', {RowsCopied},null,null,null,'{LdSrtTimenrws}','{LdEndTimenrws}',null,'slv','{wtrmrk}', '{LdEndTimenrws}')"""
            spark.sql(log_qry_no_rws)

            err_rw_cnt = initial_row_count - cleaned_row_count
            log_rule_execution = f"""INSERT INTO EDP_DEV.data_quality_rule_execution_log (lob_nm,src_nm,entity_nm,rule_nm,rule_qry_exec,err_rw_cnt,exec_lg_dt_time) VALUES (null, null, '{tbl_nm}', null, null, '{err_rw_cnt}', '{LdEndTimenrws}')"""
        else:
            # Perform incremental load
            wtrmrkbrzqry = f'SELECT MAX(watermark) AS watermark FROM {tbl_brnz_nm}'
            wtrmrkbrz = spark.sql(wtrmrkbrzqry).first()['watermark']
            print("wtrmrkbrz", wtrmrkbrz)
            wtrmrkslvqry = f'SELECT MAX(watermark) AS watermark FROM {table_slv}'
            wtrmrkslv = spark.sql(wtrmrkslvqry).first()['watermark']
            print("wtrmrkslv", wtrmrkslv)
            data_qry = f"""SELECT * FROM {tbl_brnz_nm} WHERE watermark > '{wtrmrkslv}' AND watermark <= '{wtrmrkbrz}'"""

            slv_df = spark.sql(data_qry)
            initial_row_count = slv_df.count()
            
            if slv_df.isEmpty():
                LdEndTimenrws=datetime.now().isoformat()
                log_qry_no_rws = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)VALUES (null, '{tbl_nm}', null,null, 0, 'Failure', 0,null,null,null,'{LdSrtTimenrws}','{LdEndTimenrws}',null,'slv','{wtrmrk}', '{LdEndTimenrws}')"""
                spark.sql(log_qry_no_rws)

            slv_df = spark.sql(rule_qry)
            cleaned_row_count = slv_df.count()
            window_spec = Window.partitionBy(SourceColumnKey).orderBy(col("watermark").desc())
            slv_df = slv_df.withColumn("row_num", row_number().over(window_spec)).filter(col("row_num") == 1).drop("row_num")
            print("cleaned slv_df")

            slvdeltatbl = DeltaTable.forName(spark, table_slv)
            slvnm = 'cpath'
            brznm = 'dpath'
            condtn = f"{slvnm}.{SourceColumnKey} = {brznm}.{SourceColumnKey}"

            slvdeltatbl = DeltaTable.forName(spark, table_slv)
            slvnm = 'cpath'
            brznm = 'dpath'
            condtn = f"{slvnm}.{SourceColumnKey} = {brznm}.{SourceColumnKey}"

            slvdeltatbl.alias(slvnm).merge(
                slv_df.alias(brznm), condtn
            ).whenMatchedUpdate(
                condition=f"{brznm}.watermark > {slvnm}.watermark",
                set={
                    "Emp_Id": col(f"{brznm}.Emp_Id"),
                    "Emp_nm": col(f"{brznm}.Emp_nm"),
                    "Dept_Id": col(f"{brznm}.Dept_Id"),
                    "createdt": col(f"{brznm}.createdt"),
                    "modifieddt": col(f"{brznm}.modifieddt"),
                    "watermark": greatest(col(f"{slvnm}.watermark"), col(f"{brznm}.watermark"))
                }
            ).whenNotMatchedInsert(
                values={
                    "Emp_Id": col(f"{brznm}.Emp_Id"),
                    "Emp_nm": col(f"{brznm}.Emp_nm"),
                    "Dept_Id": col(f"{brznm}.Dept_Id"),
                    "createdt": col(f"{brznm}.createdt"),
                    "modifieddt": col(f"{brznm}.modifieddt"),
                    "watermark": col(f"{brznm}.watermark")
                }
            ).execute()

            DataRead= initial_row_count
            RowsCopied = cleaned_row_count
            LdEndTimenrws=datetime.now().isoformat()
            ExecutionStatus='Success'
            log_qry_no_rws = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)VALUES (null, '{tbl_nm}', null,null, {DataRead},'{ExecutionStatus}', {RowsCopied},null,null,null,'{LdSrtTimenrws}','{LdEndTimenrws}',null,'slv','{wtrmrk}', '{LdEndTimenrws}')"""
            spark.sql(log_qry_no_rws)

            err_rw_cnt = initial_row_count - cleaned_row_count
            log_rule_execution = f"""INSERT INTO EDP_DEV.data_quality_rule_execution_log (lob_nm,src_nm,entity_nm,rule_nm,rule_qry_exec,err_rw_cnt,exec_lg_dt_time) VALUES (null, null, '{tbl_nm}', null, null, '{err_rw_cnt}', '{LdEndTimenrws}')"""


    except Exception as e:
        print(e)

In [0]:
# qry = f"""
# SELECT ilc.src_nm,ilc.entity_nm,ilc.wtr_mrk_clm_nm,ilc.wtr_mrk_clm_val,ilc.part_clm_nm,ilc.part_clm_val,ilc.interval,ilc.frequency,ilc.clm_dtls,ilc.ref_qry,ilc.src_wtr_mrk_qry,ilc.cfg_upd_wtr_mrk_qry,ilc.batch_sz,ilc.ld_seq,ilc.uc_nm,ilc.sch_nm,ilc.mig_flag,dqr.lob_nm,dqr.rule_nm,dqr.rule_typ,dqr.rule_bs_qry,dqr.rule_qry,dqr.vw_nm,dqr.mig_flag AS dqr_mig_flag
# FROM 
#     edp_dev.incremental_load_cfg ilc
# FULL OUTER JOIN
#     edp_dev.data_quality_rule dqr
# ON 
#     ilc.entity_nm = dqr.entity_nm
# """

# df_cfg = spark.sql(qry)
# dic = df_cfg.collect()
# dic = [row.asDict() for row in dic]
# print(dic)


In [0]:
qry=f"""select * from edp_dev.incremental_load_cfg """
d_qry=f"""select * from edp_dev.data_quality_rule """
df_cfg2=spark.sql(d_qry)
df_cfg1=spark.sql(qry)
df_cfg2.select(col('lob_nm'),col('src_nm'),col('entity_nm'),col('rule_nm'),col('rule_typ'),col('rule_bs_qry'),col('rule_qry'),col('vw_nm'),col('mig_flag'))
df_cfg1.select(col('src_nm'),col('entity_nm'),col('wtr_mrk_clm_nm'),col('wtr_mrk_clm_val'),col('part_clm_nm'),col('part_clm_val'),col('interval'),col('frequency'), col('clm_dtls'),col('ref_qry'),col('src_wtr_mrk_qry'),col('cfg_upd_wtr_mrk_qry'),col('batch_sz'),col('ld_seq'),col('uc_nm'),col('sch_nm'),col('mig_flag'))

df_cfg = df_cfg1.join(df_cfg2, on=['entity_nm'], how='outer')
dic = df_cfg.collect()
dic = [row.asDict() for row in dic]
print(dic)

[{'entity_nm': 'src_emp', 'src_nm': '', 'wtr_mrk_clm_nm': 'modifieddt', 'wtr_mrk_clm_val': '2024-07-01T09:01:12.663955', 'src_clm_key': 'Emp_ID', 'part_clm_nm': '', 'part_clm_val': '', 'interval': 1, 'frequency': 1, 'clm_dtls': '(Select Emp_Id, Emp_nm, Dept_Id, createdt, modifieddt, modifieddt as watermark from EDP_source.src_emp where modifieddt > "{}"  and modifieddt <="{}")', 'ref_qry': '', 'src_wtr_mrk_qry': 'select max("{}") from EDP_source."{}"', 'cfg_upd_wtr_mrk_qry': 'update EDP_DEV.incremental_load_cfg set wtr_mrk_clm_val="{}" where entity_nm="{}"', 'batch_sz': '', 'ld_seq': None, 'uc_nm': '', 'sch_nm': '', 'mig_flag': True, 'lob_nm': '', 'rule_nm': 'NULL_CHECK', 'rule_typ': 'Data quality check', 'rule_bs_qry': '', 'rule_qry': 'SELECT * from edp_dev.src_emp_brnz WHERE Emp_Id IS NOT NULL AND Emp_nm IS NOT NULL AND Dept_Id IS NOT NULL AND createdt IS NOT NULL AND modifieddt IS NOT NULL', 'vw_nm': ''}, {'entity_nm': 'src_emp', 'src_nm': '', 'wtr_mrk_clm_nm': 'modifieddt', 'wtr_mr

In [0]:
from threading import Thread
from queue import Queue

q = Queue()

worker_count = worker_count
def run_tasks(function, q):
    while not q.empty():
        try:
            value = q.get()
            function(value)
        except Exception as e:
            output=f'{e}'
            print(output)
    
        finally:
            q.task_done()

for table in  dic:
   q.put(table)

In [0]:
for i in range(driver_count):
    try:
        t=Thread(target=run_tasks, args=(load_table, q))
        t.daemon = True
        t.start()
    except Exception as e: 
        output=f'{e}'
        print(output)

q.join()

wtrmrkbrz 2024-07-01T09:01:12.663955
wtrmrkbrz 2024-07-01T09:01:12.663955
wtrmrkbrz 2024-07-01T09:01:12.663955
wtrmrkslv 2024-07-01T09:01:12.663955
wtrmrkslv 2024-07-01T09:01:12.663955
wtrmrkslv 2024-07-01T09:01:12.663955
cleaned slv_df
cleaned slv_df
cleaned slv_df
